In [2]:
import json
import time

import gurobipy as gp
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib_inline
import numpy as np
import pandas as pd
from gurobipy import GRB
from tqdm.auto import tqdm

from orc.branch import (
    branch_beasley, branch_reduced_costs, 
    branch_lp
)
from orc.callbacks import (
    ColumnInclusionCallback,
    LagrPenaltiesReductionCallback,
    PrimalHeurCallback
)
from orc.data_structures import (
    BranchAndBound, TimeLimitException
)
from orc.primal import dobson, greedy, hall_hochbaum
from orc.relaxation import subgrad_opt, lp_rel
from orc.utils import generate_problem

mpl.rcParams["font.family"] = ["serif"]
mpl.rcParams["font.serif"] = ["cmr10"]
matplotlib_inline.backend_inline.set_matplotlib_formats("retina")

# Primal heuristics comparisons

## Without fixed variables

In [ ]:
table = []
funcs = [(greedy, "Greedy"), (dobson, "Dobson"),
         (hall_hochbaum, "Hall-Hochbaum")]
for rows, cols in tqdm(
    [(5, 10), (10, 20), (20, 50), (50, 100)], leave=False):
    res = {"Rows": rows, "Cols": cols, "Greedy": 0, 
           "Dobson": 0, "Hall-Hochbaum": 0}
    for i in tqdm(range(10)):
        A, b = generate_problem(rows, cols)
        v = []
        for f, name in funcs:
            ub = np.sum(A, axis=0) @ f(A, b, [], [])
            v.append(ub)
        j = np.argmin(v)
        best = funcs[j][1]
        res[best] += 1
    table.append(res)

In [ ]:
with open("results/primal.json", "w") as f:
    out = {"output": table}
    json.dump(out, f)

In [ ]:
with open("results/primal.json", "r") as f:
    table = json.load(f)["output"]

In [ ]:
df = pd.DataFrame(table)
df = df.set_index(["Rows", "Cols"])
df

In [ ]:
s = df.style.highlight_max(axis=1, props="bfseries: ;")
print(s.to_latex(column_format="llccc", 
                 hrules=True, 
                 position_float="centering"))

## With fixed variables

In [ ]:
np.random.seed(42)

table = []
funcs = [(greedy, "Greedy"), (dobson, "Dobson"),
         (hall_hochbaum, "Hall-Hochbaum")]
for rows, cols in tqdm(
    [(5, 10), (10, 20), (20, 50), (50, 100)], leave=False):
    res = {"Rows": rows, "Cols": cols, "Greedy": 0, 
           "Dobson": 0, "Hall-Hochbaum": 0}
    for i in tqdm(range(10)):
        A, b = generate_problem(rows, cols)
        fixed_n = int(cols * 0.3) 
        fixed = np.random.choice(
            A.shape[-1], fixed_n, replace=False)
        x0 = fixed[:int(fixed_n / 2)]
        x1 = fixed[int(fixed_n / 2) + 1:]
        x = np.ones(A.shape[-1])
        x[x0] = 0
        if np.any(A @ x < b):
            continue
        v = []
        for f, name in funcs:
            ub = np.sum(A, axis=0) @ f(A, b, [], [])
            v.append(ub)
        j = np.argmin(v)
        best = funcs[j][1]
        res[best] += 1
    table.append(res)

In [ ]:
table

# Subgradient optimization lower bounds

In [ ]:
data = {}
for rows, cols in tqdm(
    [(10, 20), (20, 50), (40, 80), (80, 150)], leave=False):
    A, b = generate_problem(rows, cols)
    x = hall_hochbaum(A, b, [], [])
    ub = np.sum(A, axis=0) @ x

    res = []
    for omega in [10, 20, 40, 50, 70, 100, 200, 300]:
        lb = subgrad_opt(A, b, ub, [], [], omega=omega)
        res.append([omega, lb])
    data[(rows, cols)] = res

In [ ]:
fig, ax = plt.subplots()
for k, v in data.items():
    x = np.array(v)[:,0]
    y = np.array(v)[:,1]
    y = (y - y.min()) / (y.max() - y.min())
    ax.plot(x, y, label=f"{k[0]} rows, {k[1]} cols")
ax.legend()
ax.set_xlabel("Iterations")
ax.set_ylabel("Scaled lower bound")
ax.set_title("Subgradient optimization lower bounds")
plt.savefig("report/img/subgrad_lb.png", 
            dpi=300, bbox_inches="tight")

# Branch and Bound models

In [2]:
lagr_callback = LagrPenaltiesReductionCallback()
col_callback = ColumnInclusionCallback()
primal_heur = PrimalHeurCallback()
primal_heur_root = PrimalHeurCallback(only_root=True)

models = {
    "Subgrad": {
        "branch_strategy": branch_reduced_costs, 
        "lb_strategy": subgrad_opt
        },
    "SubgradPrimal": {
        "branch_strategy": branch_reduced_costs, 
        "lb_strategy": subgrad_opt,
        "callbacks": [primal_heur]
        },
    "SubgradPrimalRed": {
        "branch_strategy": branch_reduced_costs, 
        "lb_strategy": subgrad_opt,
        "callbacks": [primal_heur, lagr_callback, col_callback]
        },
    "SubgradPrimalRootRed": {
        "branch_strategy": branch_reduced_costs, 
        "lb_strategy": subgrad_opt,
        "callbacks": [primal_heur_root, lagr_callback, col_callback]
        },
    "SubgradPrimalRedBeasleyBranch": {
        "branch_strategy": branch_beasley, 
        "lb_strategy": subgrad_opt,
        "callbacks": [primal_heur, lagr_callback, col_callback]
        },
    "LPPrimalRed": {
        "branch_strategy": branch_lp, 
        "lb_strategy": lp_rel,
        "callbacks": [primal_heur, col_callback]}
}
short_names = {
    "Gurobi": "Gurobi",
    "Subgrad": "S",
    "SubgradPrimal": "SP",
    "SubgradPrimalRed": "SPR",
    "SubgradPrimalRootRed": "SPRR",
    "SubgradPrimalRedBeasleyBranch": "SPRB",
    "LPPrimalRed": "LP"
}
col_names = ["S", "SP", "SPR", "SPRR", "SPRB", "LP"]

In [ ]:
data = {}
time_limit = 60 * 5
for rows, cols, density in tqdm(
    [(5, 10, 0.3), (5, 10, 0.5), (5, 10, 0.7), 
     (10, 20, 0.3), (10, 20, 0.5), (10, 20, 0.7),
     (13, 22, 0.3), (13, 22, 0.5), (13, 22, 0.7),
     (15, 25, 0.3), (15, 25, 0.5), (15, 25, 0.7)],
     desc="Problem:", leave=False):
    res = {}
    A, b = generate_problem(rows, cols, density)
    
    start = time.process_time()
    m = gp.Model()
    m.Params.LogToConsole = 0
    x = m.addMVar(A.shape[-1], vtype=GRB.BINARY, name="x")
    m.setObjective(np.sum(A, axis=0) @ x)
    m.addConstr(A @ x >= b)
    m.optimize()
    elapsed = time.process_time() - start
    opt_gurobi = m.getObjective().getValue()
    
    x = []
    for v in m.getVars():
        x.append(v.x)
    x = np.array(x)
    
    # Make sure that the solution is feasible
    assert np.all(A @ x >= b)

    res["Gurobi"] = (elapsed, m.NodeCount)

    for model, config in tqdm(
        models.items(), desc="Model:", leave=False):
        start = time.process_time()
        bb = BranchAndBound(**config, time_start=start, 
                            time_limit=time_limit)
        try:
            bb.search(A, b)
            elapsed = time.process_time() - start
            x = np.zeros(A.shape[-1])
            x[bb.best.x1] = 1
            
            # Make sure that the solution is feasible
            assert np.all(A @ x >= b)
            opt = np.sum(A, axis=0) @ x

            # Check that the solution is optimal
            assert opt == opt_gurobi
            res[model] = (elapsed, bb.node_count)
        except TimeLimitException:
            res[model] = (np.nan, np.nan)
        
    data[(rows, cols, density)] = res

In [ ]:
with open("results/models.json", "w") as f:
    nd = {str(k): v for k, v in data.items()}
    json.dump(nd, f)

In [3]:
with open("results/models.json", "r") as f:
    results = json.load(f)

In [4]:
time_dict = {
    tuple(k[1:-1].split(", ")): {
        short_names[model]: values[0] for model, values in v.items()}  
    for k, v in results.items()}
df = pd.DataFrame(time_dict)
df = df.T.round(2)
df = df.rename_axis(["Rows", "Cols", "Density"])
df

Gurobi      S      SP     SPR    SPRR    SPRB      LP
Rows Cols Density                                                       
5    10   0.3        0.05   0.00    0.00    0.00    0.00    0.00    0.02
          0.5        0.00   0.03    0.08    0.00    0.00    0.05    0.06
          0.7        0.00   0.02    0.02    0.02    0.02    0.06    0.14
10   20   0.3        0.00   4.86   16.27    4.17    2.91    1.64   31.47
          0.5        0.02  12.47   20.56    7.38    4.22    9.66   68.22
          0.7        0.00  29.30   36.59   18.12   12.05    9.81  118.48
13   22   0.3        0.00  20.84   33.45    7.59    5.36    3.55   64.22
          0.5        0.02  29.53   38.27   17.14   14.64   10.72  288.70
          0.7        0.06  94.36  116.97   43.14   29.98   42.47     NaN
15   25   0.3        0.00  62.98   93.23   24.17   17.77    9.36     NaN
          0.5        0.00    NaN     NaN  161.31  113.97   38.22     NaN
          0.7        0.06    NaN     NaN  291.48  248.05  127.39     NaN

In [5]:
s = df.style.format("{:.2f}") \
            .highlight_min(subset=col_names, 
                           axis=1, props="bfseries: ;")
print(s.to_latex(column_format="lllccccccc", 
                 hrules=True, 
                 clines="skip-last;data",
                 caption="Runtime of different configurations of the branch-and-bound algorithm over randomly generated TRAP instances with different numbers of rows and columns and different densities. Configurations whose runtime exceeded 5 minutes were assigned nan values.",
                 label="tab:results:runtime", 
                 position_float="centering"))

\begin{table}
\centering
\begin{tabular}{lllccccccc}
\toprule
 &  &  & Gurobi & S & SP & SPR & SPRR & SPRB & LP \\
Rows & Cols & Density &  &  &  &  &  &  &  \\
\midrule
\multirow[c]{3}{*}{5} & \multirow[c]{3}{*}{10} & 0.3 & 0.05 & \bfseries 0.00 & \bfseries 0.00 & \bfseries 0.00 & \bfseries 0.00 & \bfseries 0.00 & 0.02 \\
 &  & 0.5 & 0.00 & 0.03 & 0.08 & \bfseries 0.00 & \bfseries 0.00 & 0.05 & 0.06 \\
 &  & 0.7 & 0.00 & \bfseries 0.02 & \bfseries 0.02 & \bfseries 0.02 & \bfseries 0.02 & 0.06 & 0.14 \\
\cline{1-10} \cline{2-10}
\multirow[c]{3}{*}{10} & \multirow[c]{3}{*}{20} & 0.3 & 0.00 & 4.86 & 16.27 & 4.17 & 2.91 & \bfseries 1.64 & 31.47 \\
 &  & 0.5 & 0.02 & 12.47 & 20.56 & 7.38 & \bfseries 4.22 & 9.66 & 68.22 \\
 &  & 0.7 & 0.00 & 29.30 & 36.59 & 18.12 & 12.05 & \bfseries 9.81 & 118.48 \\
\cline{1-10} \cline{2-10}
\multirow[c]{3}{*}{13} & \multirow[c]{3}{*}{22} & 0.3 & 0.00 & 20.84 & 33.45 & 7.59 & 5.36 & \bfseries 3.55 & 64.22 \\
 &  & 0.5 & 0.02 & 29.53 & 38.27 & 17.14 & 14.64 

In [6]:
nodes_dict = {
    tuple(k[1:-1].split(", ")): {
        short_names[model]: values[1] for model, values in v.items()}  
    for k, v in results.items()}
df = pd.DataFrame(nodes_dict)
df = df.T
df = df.rename_axis(["Rows", "Cols", "Density"])
df

Gurobi         S        SP       SPR      SPRR      SPRB  \
Rows Cols Density                                                             
5    10   0.3         0.0      33.0      21.0       5.0       5.0       7.0   
          0.5         1.0     211.0     203.0      49.0      51.0      55.0   
          0.7         1.0     263.0     253.0      63.0      71.0      49.0   
10   20   0.3         1.0   19973.0   39189.0    6313.0    6323.0    2231.0   
          0.5        50.0   54737.0   50515.0    9833.0   10431.0   15731.0   
          0.7        25.0   97731.0   80731.0   21107.0   22757.0   13049.0   
13   22   0.3         1.0   85147.0   82659.0    9889.0   12159.0    4641.0   
          0.5         1.0  107217.0   85605.0   25719.0   33459.0   11809.0   
          0.7         1.0  255533.0  202681.0   39033.0   44051.0   38831.0   
15   25   0.3         1.0  183249.0  145459.0   21561.0   27331.0    7761.0   
          0.5        59.0       NaN       NaN  151895.0  180799.0   35241.0   
          0.7       384.0       NaN       NaN  266983.0  365631.0  113737.0   

                         LP  
Rows Cols Density            
5    10   0.3          23.0  
          0.5         183.0  
          0.7         323.0  
10   20   0.3       55541.0  
          0.5      124795.0  
          0.7      209799.0  
13   22   0.3      117271.0  
          0.5      416059.0  
          0.7           NaN  
15   25   0.3           NaN  
          0.5           NaN  
          0.7           NaN

In [7]:
s = df.style.format("{:.0f}") \
            .highlight_min(subset=col_names, 
                           axis=1, props="bfseries: ;")
print(s.to_latex(column_format="lllccccccc", 
                 hrules=True, 
                 clines="skip-last;data",
                 caption="Number of nodes generated by different configurations of the branch-and-bound algorithm over randomly generated TRAP instances with different numbers of rows and columns and different densities. Configurations whose runtime exceeded 5 minutes were assigned nan values.",
                 label="tab:results:nodes", 
                 position_float="centering"))

\begin{table}
\centering
\begin{tabular}{lllccccccc}
\toprule
 &  &  & Gurobi & S & SP & SPR & SPRR & SPRB & LP \\
Rows & Cols & Density &  &  &  &  &  &  &  \\
\midrule
\multirow[c]{3}{*}{5} & \multirow[c]{3}{*}{10} & 0.3 & 0 & 33 & 21 & \bfseries 5 & \bfseries 5 & 7 & 23 \\
 &  & 0.5 & 1 & 211 & 203 & \bfseries 49 & 51 & 55 & 183 \\
 &  & 0.7 & 1 & 263 & 253 & 63 & 71 & \bfseries 49 & 323 \\
\cline{1-10} \cline{2-10}
\multirow[c]{3}{*}{10} & \multirow[c]{3}{*}{20} & 0.3 & 1 & 19973 & 39189 & 6313 & 6323 & \bfseries 2231 & 55541 \\
 &  & 0.5 & 50 & 54737 & 50515 & \bfseries 9833 & 10431 & 15731 & 124795 \\
 &  & 0.7 & 25 & 97731 & 80731 & 21107 & 22757 & \bfseries 13049 & 209799 \\
\cline{1-10} \cline{2-10}
\multirow[c]{3}{*}{13} & \multirow[c]{3}{*}{22} & 0.3 & 1 & 85147 & 82659 & 9889 & 12159 & \bfseries 4641 & 117271 \\
 &  & 0.5 & 1 & 107217 & 85605 & 25719 & 33459 & \bfseries 11809 & 416059 \\
 &  & 0.7 & 1 & 255533 & 202681 & 39033 & 44051 & \bfseries 38831 & nan \\
\cline{1-10}